# Designing and Creating a Baseball Database

## We will be working with four csv files describing Major League Baseball game statistics from Retrosheet:
* game_log.csv
* park_codes.csv
* person_codes.csv
* team_codes.csv

### Explanation of fields in game_log.csv:

In [1]:
# read in from command prompt
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


## Read in the data:

In [2]:
import pandas as pd
pd.set_option('max_columns',180)
pd.set_option('max_rows',200000)
pd.set_option('max_colwidth',5000)

game_log = pd.read_csv('game_log.csv')
park_codes = pd.read_csv('park_codes.csv')
person_codes = pd.read_csv('person_codes.csv')
team_codes = pd.read_csv('team_codes.csv')

/dataquest/system/env/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (12,13,14,15,19,20,81,82,83,84,85,86,87,88,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Preliminary data exploration:

### game_log.csv

In [3]:
game_log.shape

(171907, 161)

In [4]:
game_log.head(15)

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,2,54.0,D,NaN,NaN,NaN,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,6.0,1.0,NaN,-1.0,NaN,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,NaN,-1.0,NaN,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,NaN,NaN,NaN,NaN,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,NaN,1,WS3,NaN,1,20,18,54.0,D,NaN,NaN,NaN,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,NaN,5.0,3.0,NaN,-1.0,NaN,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,NaN,2.0,1.0,NaN,-1.0,NaN,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,NaN,NaN,NaN,NaN,spala101,Al Spalding,braia102,Asa Brainard,wrigg101,George Wright,6.0,barnr102,Ross

In [5]:
new_col = ( game_log["date"].astype(str) 
           + game_log["v_name"]
           + game_log["h_name"]
           + game_log["h_game_number"].astype(str)
          )
len(pd.unique(new_col))

171907

In [6]:
len(game_log["park_id"].dropna().unique())

245

In [7]:
game_log.describe(include="all")

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
count,1.719070e+05,171907.000000,171907,171907,170821,171907.000000,171907,170821,171907.000000,171907.000000,171907.000000,140841.000000,140150,116,145,180,171907,118877.000000,136701.000000,147271,147271,140838.000000,140838.000000,140772.000000,140835.000000,140800.000000,139488.000000,140838.000000,135885.000000,140838.000000,140728.000000,113541.000000,140746.000000,140775.000000,127317.000000,131222.000000,134357.000000,140838.000000,140838.000000,138135.000000,140668.000000,140624.000000,140838.000000,140829.000000,140837.000000,140807.000000,140801.000000,140825.000000,140838.000000,140838.000000,140838.000000,140772.000000,140835.000000,140800.000000,139515.000000,140838.000000,135885.000000,140838.000000,140730.000000,113541.000000,140746.000000,140775.000000,127317.000000,131222.000000,134357.000000,140838.000000,140838.000000,138117.000000,140683.000000,140618.000000,140838.000000,140829.000000,140837.000000,140808.000000,140800.000000,140824.000000,140838.000000,171888,171888,147040,147040,88540,88540,116723,116723,203,203,9,9,171907,171907,171907,171907,140229,140229,140229,140229,48018,48018,105699,105699,171863,171863,171863,171863,140838,140838,140838.000000,140838,140838,140838.000000,140838,140838,140838.000000,140838,140838,140838.000000,140838,140838,140838.000000,140838,140838,140838.000000,140838,140838,140838.000000,140838,140838,140838.000000,140835,140835,140835.000000,140838,140838,140838.000000,140838,140838,140838.000000,140838,140838,140838.000000,140838,140838,140838.000000,140838,140838,140838.000000,140838,140838,140838.00

### Game_log.csv contains game details and other statistics (offensive/defensive/pitching) on baseball games from 1871 to 2016. It is unique by game date, visiting team, home team, and home team game number (but there is no natural primary key). There are missing values and some non-sensical values (e.g. negative values for v_triples, v_homeruns, etc.).

### park_codes.csv

In [8]:
park_codes.shape

(252, 9)

In [9]:
park_codes.head()

,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,NaN,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,NaN,Altoona,PA,04/30/1884,05/31/1884,UA,NaN
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,04/19/1966,NaN,AL,NaN
3,ARL01,Arlington Stadium,NaN,Arlington,TX,04/21/1972,10/03/1993,AL,NaN
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,04/11/1994,NaN,AL,NaN


In [10]:
len(park_codes["park_id"].unique())

252

In [11]:
park_codes.describe(include="all")

,park_id,name,aka,city,state,start,end,league,notes
count,252,252,58,252,252,252,222,186,128
unique,252,241,56,85,36,215,205,6,123
top,TOL01,Athletic Park,Beyerle's Park,Philadelphia,NY,05/01/1883,10/03/1915,NL,CL3:7/22&7/29&8/26&9/2/1888
freq,1,4,2,14,40,5,3,88,2


### Park_codes.csv contains infromation like the location and operation dates for 252 baseball parks. This dataset can be related to the main game_log.csv dataset using the park_id variable.

### person_codes.csv

In [12]:
person_codes.shape

(20494, 7)

In [13]:
person_codes.head()

,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,04/06/2004,NaN,NaN,NaN
1,aaroh101,Aaron,Hank,04/13/1954,NaN,NaN,NaN
2,aarot101,Aaron,Tommie,04/10/1962,NaN,04/06/1979,NaN
3,aased001,Aase,Don,07/26/1977,NaN,NaN,NaN
4,abada001,Abad,Andy,09/10/2001,NaN,NaN,NaN


In [14]:
person_codes.describe(include="all")

,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
count,20494,20494,20433,19183,701,1566,1546
unique,20494,10397,2484,10278,537,332,1130
top,brazc001,Smith,Bill,05/01/1884,05/01/1884,04/01/2013,08/25/1978
freq,1,168,579,36,10,29,52


### Person_codes.csv includes information on each player/coach/umpire debut date. The main dataset (game_log.csv) contains player, pitcher, and umpire names and IDs for the home and visiting teams, which can be merged with the id, last, and first variables from person_codes.csv.

### team_codes.csv

In [15]:
team_codes.shape

(150, 8)

In [16]:
team_codes.head()

,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1
1,ARI,NL,1998,0,Arizona,Diamondbacks,ARI,1
2,BFN,NL,1879,1885,Buffalo,Bisons,BFN,1
3,BFP,PL,1890,1890,Buffalo,Bisons,BFP,1
4,BL1,NaN,1872,1874,Baltimore,Canaries,BL1,1


In [17]:
sorted(team_codes["team_id"].unique())

['ALT',
 'ANA',
 'ARI',
 'ATL',
 'BAL',
 'BFN',
 'BFP',
 'BL1',
 'BL2',
 'BL4',
 'BLA',
 'BLF',
 'BLN',
 'BLU',
 'BOS',
 'BR1',
 'BR2',
 'BR3',
 'BR4',
 'BRF',
 'BRO',
 'BRP',
 'BS1',
 'BS2',
 'BSN',
 'BSP',
 'BSU',
 'BUF',
 'CAL',
 'CH1',
 'CH2',
 'CHA',
 'CHF',
 'CHN',
 'CHP',
 'CHU',
 'CIN',
 'CL1',
 'CL2',
 'CL3',
 'CL4',
 'CL5',
 'CL6',
 'CLE',
 'CLP',
 'CN1',
 'CN2',
 'CN3',
 'CNU',
 'COL',
 'DET',
 'DTN',
 'ELI',
 'FLO',
 'FW1',
 'HAR',
 'HOU',
 'HR1',
 'IN1',
 'IN2',
 'IN3',
 'IND',
 'KC1',
 'KC2',
 'KCA',
 'KCF',
 'KCN',
 'KCU',
 'KEO',
 'LAA',
 'LAN',
 'LS1',
 'LS2',
 'LS3',
 'MIA',
 'MID',
 'MIL',
 'MIN',
 'ML2',
 'ML3',
 'MLA',
 'MLN',
 'MLU',
 'MON',
 'NEW',
 'NH1',
 'NY1',
 'NY2',
 'NY3',
 'NY4',
 'NYA',
 'NYN',
 'NYP',
 'OAK',
 'PH1',
 'PH2',
 'PH3',
 'PH4',
 'PHA',
 'PHI',
 'PHN',
 'PHP',
 'PHU',
 'PIT',
 'PRO',
 'PT1',
 'PTF',
 'PTP',
 'PTU',
 'RC1',
 'RC2',
 'RIC',
 'SDN',
 'SE1',
 'SEA',
 'SFN',
 'SL1',
 'SL2',
 'SL3',
 'SL4',
 'SL5',
 'SLA',
 'SLF',
 'SLN',
 'SLU',


In [18]:
sorted(game_log["v_name"].unique())

['ALT',
 'ANA',
 'ARI',
 'ATL',
 'BAL',
 'BFN',
 'BFP',
 'BL1',
 'BL2',
 'BL4',
 'BLA',
 'BLF',
 'BLN',
 'BLU',
 'BOS',
 'BR1',
 'BR2',
 'BR3',
 'BR4',
 'BRF',
 'BRO',
 'BRP',
 'BS1',
 'BS2',
 'BSN',
 'BSP',
 'BSU',
 'BUF',
 'CAL',
 'CH1',
 'CH2',
 'CHA',
 'CHF',
 'CHN',
 'CHP',
 'CHU',
 'CIN',
 'CL1',
 'CL2',
 'CL3',
 'CL4',
 'CL5',
 'CL6',
 'CLE',
 'CLP',
 'CN1',
 'CN2',
 'CN3',
 'CNU',
 'COL',
 'DET',
 'DTN',
 'ELI',
 'FLO',
 'FW1',
 'HAR',
 'HOU',
 'HR1',
 'IN1',
 'IN2',
 'IN3',
 'IND',
 'KC1',
 'KC2',
 'KCA',
 'KCF',
 'KCN',
 'KCU',
 'KEO',
 'LAA',
 'LAN',
 'LS1',
 'LS2',
 'LS3',
 'MIA',
 'MID',
 'MIL',
 'MIN',
 'ML2',
 'ML3',
 'MLA',
 'MLN',
 'MON',
 'NEW',
 'NH1',
 'NY1',
 'NY2',
 'NY3',
 'NY4',
 'NYA',
 'NYN',
 'NYP',
 'OAK',
 'PH1',
 'PH2',
 'PH3',
 'PH4',
 'PHA',
 'PHI',
 'PHN',
 'PHP',
 'PHU',
 'PIT',
 'PRO',
 'PT1',
 'PTF',
 'PTP',
 'RC1',
 'RC2',
 'RIC',
 'SDN',
 'SE1',
 'SEA',
 'SFN',
 'SL1',
 'SL2',
 'SL3',
 'SL4',
 'SL5',
 'SLA',
 'SLF',
 'SLN',
 'SLU',
 'SPU',
 'SR1',


In [19]:
team_codes.describe(include="all")

,team_id,league,start,end,city,nickname,franch_id,seq
count,150,124,150.000000,150.000000,150,150,150,150.000000
unique,149,6,NaN,NaN,54,93,115,NaN
top,MIL,NL,NaN,NaN,Washington,Reds,BS1,NaN
freq,2,45,NaN,NaN,12,6,4,NaN
mean,NaN,NaN,1902.733333,1517.440000,NaN,NaN,NaN,1.293333
std,NaN,NaN,37.326002,761.772866,NaN,NaN,NaN,0.585584
min,NaN,NaN,1871.000000,0.000000,NaN,NaN,NaN,1.000000
25%,NaN,NaN,1879.000000,1872.000000,NaN,NaN,NaN,1.000000
50%,NaN,NaN,1887.000000,1884.000000,NaN,NaN,NaN,1.000000
75%,NaN,NaN,1911.250000,1891.000000,NaN,NaN,NaN,1.000000


In [20]:
team_codes[team_codes.duplicated(subset="team_id", keep=False) == True]

,team_id,league,start,end,city,nickname,franch_id,seq
112,MIL,AL,1970,1997,Milwaukee,Brewers,SE1,2
113,MIL,NL,1998,0,Milwaukee,Brewers,SE1,3


### Team_codes.csv contains nicknames, duration, the location, and the league for different baseball teams. (Note that one baseball team_id=MIL is repeated twice in this dataset due to a change in its league.) This dataset can be merged to the variables v_name and h_name in the main game_log.csv dataset, using the team_id variable in team_codes.csv.  

## Start creating the SQL database

In [21]:
import sqlite3

db = 'mlb.db'

def run_query(query):
    '''takes in a SQL query and returns a pandas dataframe of the query'''
    with sqlite3.connect(db) as conn:
        return pd.read_sql(query,conn)
    
def run_command(command):
    '''takes in a SQL command and executes it'''
    with sqlite3.connect(db) as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(command)
        
def show_tables():
    '''calls run_query to return a list of all tables and views in the database'''
    query = '''SELECT name, type 
               FROM sqlite_master 
               WHERE type IN ('table','view');'''
    return run_query(query)

In [22]:
with sqlite3.connect(db) as conn:
    game_log.to_sql('game_log',conn,index=False,if_exists='replace')
    park_codes.to_sql('park_codes',conn,index=False,if_exists='replace')
    person_codes.to_sql('person_codes',conn,index=False,if_exists='replace')
    team_codes.to_sql('team_codes',conn,index=False,if_exists='replace')

In [23]:
show_tables()

,name,type
0,person,table
1,park,table
2,league,table
3,game,table
4,team,table
5,team_appearance,table
6,appearance_type,table
7,person_appearance,table
8,game_log,table
9,park_codes,table


In [24]:
# add new variable to game_log to be used as the primary key

c = '''
    ALTER TABLE game_log
    ADD COLUMN game_id TEXT;
    '''

run_command(c)

c = '''
    UPDATE game_log
    SET game_id = h_name || date || number_of_game;
    '''

run_command(c)

In [25]:
# check that the variable creation looks okay

query = '''
    SELECT game_id, h_name, date, number_of_game
    FROM game_log
    LIMIT 5;
    '''
run_query(query)

,game_id,h_name,date,number_of_game
0,FW1187105040,FW1,18710504,0
1,WS3187105050,WS3,18710505,0
2,RC1187105060,RC1,18710506,0
3,CH1187105080,CH1,18710508,0
4,TRO187105090,TRO,18710509,0


## Looking for opportunities to Normalize the data

* player IDs, names, and def_pos's, as well as pitchers, managers, and umpires in the game_log dataset are attributes of the people, not the games. This information might be better placed in the person_codes dataset or in a separate 'game_players' dataset.

* The start and end columns in the park_codes dataset could be derived by looking at the date data in the game_log dataset, so it is redundant.

* Debut dates in the person_codes dataset could be derived by looking at the date data in the game_log dataset, so this is also redundant.

## Desiging a schema for the baseball database using [DBDesigner.net](https://www.dbdesigner.net/)

### first version:

![baseball_schema_v1](C:\Users\C\Desktop\datascience.io\baseball_schema_v1.jpg)

### final version:

![baseball_schema_v2](C:\Users\C\Desktop\datascience.io\baseball_schema_v2.jpg)

In [26]:
# create person table

c1 = '''
    CREATE TABLE IF NOT EXISTS person (
    person_id TEXT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT)'''

c2 = '''
    INSERT OR IGNORE INTO person
    SELECT  id,
            first,
            last
    FROM person_codes'''

run_command(c1)
run_command(c2)

q = '''
    SELECT *
    FROM person
    LIMIT 5'''

run_query(q)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad


In [27]:
# create park table

c1 = '''
    CREATE TABLE IF NOT EXISTS park (
    park_id TEXT PRIMARY KEY,
    name TEXT,
    nickname TEXT,
    city TEXT,
    state TEXT,
    notes TEXT)'''

c2 = '''
    INSERT OR IGNORE INTO park
    SELECT  park_id,
            name,
            aka,
            city,
            state,
            notes
    FROM park_codes'''

run_command(c1)
run_command(c2)

q = '''
    SELECT *
    FROM park
    LIMIT 5'''

run_query(q)

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None


In [28]:
# create league table

c1 = '''
    CREATE TABLE IF NOT EXISTS league (
    league_id TEXT PRIMARY KEY,
    name TEXT)'''

c2 = '''
    INSERT OR IGNORE INTO league
    VALUES
        ("NL", "National League"),
        ("AL", "American League"),
        ("AA", "American Association"),
        ("FL", "Federal League"),
        ("PL", "Players League"),
        ("UA", "Union Association")'''

run_command(c1)
run_command(c2)

q = '''
    SELECT *
    FROM league
    LIMIT 5'''

run_query(q)

,league_id,name
0,NL,National League
1,AL,American League
2,AA,American Association
3,FL,Federal League
4,PL,Players League


In [39]:
# create appearance type table

c1 = '''DROP TABLE IF EXISTS appearance_type'''

run_command(c1)

c2 = '''
CREATE TABLE appearance_type (
    appearance_type_id TEXT PRIMARY KEY,
    name TEXT,
    category TEXT
)
'''
run_command(c2)

appearance_type_csv = pd.read_csv('appearance_type.csv')

with sqlite3.connect('mlb.db') as conn:
    appearance_type_csv.to_sql('appearance_type',
                           conn,
                           index=False,
                           if_exists='append')

In [40]:
q = '''
    SELECT *
    FROM appearance_type
    '''

run_query(q)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense
5,O6,Batter 6,offense
6,O7,Batter 7,offense
7,O8,Batter 8,offense
8,O9,Batter 9,offense
9,D1,Pitcher,defense


In [41]:
# create team table

c1 = '''
    CREATE TABLE IF NOT EXISTS team (
    team_id TEXT PRIMARY KEY,
    league_id TEXT,
    city TEXT,
    nickname TEXT,
    franch_id TEXT,
    FOREIGN KEY (league_id) REFERENCES league(league_id))'''

c2 = '''
    INSERT OR IGNORE INTO team
    SELECT  team_id,
            league as league_id,
            city,
            nickname,
            franch_id
    FROM team_codes'''

run_command(c1)
run_command(c2)

q = '''
    SELECT *
    FROM team
    LIMIT 5'''

run_query(q)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1


In [42]:
# create game table

c1 = '''
    CREATE TABLE IF NOT EXISTS game (
    game_id TEXT PRIMARY KEY,
    date TEXT,
    number_of_game INTEGER,
    park_id TEXT,
    length_outs INTEGER,
    day BOOLEAN,
    completion TEXT,
    forfeit TEXT,
    protest TEXT,
    attendance INTEGER,
    length_minutes INTEGER,
    additional_info TEXT,
    acquisition_info TEXT,
    FOREIGN KEY (park_id) REFERENCES park(park_id)
    )'''

c2 = '''
    INSERT OR IGNORE INTO game
    SELECT 
        game_id,
        date,
        number_of_game,
        park_id,
        length_outs,
        CASE 
            WHEN day_night = "D" THEN 1
            WHEN day_night = "N" THEN 0
            ELSE NULL
            END 
            as day,
        completion,
        forefeit,
        protest,
        attendance,
        length_minutes,
        additional_info,
        acquisition_info
    FROM game_log'''

run_command(c1)
run_command(c2)

q = '''
    SELECT *
    FROM game
    LIMIT 5'''

run_query(q)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forfeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54,1,None,None,None,200,120,None,Y
1,WS3187105050,18710505,0,WAS01,54,1,None,None,None,5000,145,HTBF,Y
2,RC1187105060,18710506,0,RCK01,54,1,None,None,None,1000,140,None,Y
3,CH1187105080,18710508,0,CHI01,54,1,None,None,None,5000,150,None,Y
4,TRO187105090,18710509,0,TRO01,54,1,None,None,None,3250,145,HTBF,Y


In [43]:
# create team_appearance table

c1 = '''
CREATE TABLE IF NOT EXISTS team_appearance (
    team_id TEXT,
    game_id TEXT,
    home BOOLEAN,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats INTEGER,
    hits INTEGER,
    doubles INTEGER,
    triples INTEGER,
    homeruns INTEGER,
    rbi INTEGER,
    sacrifice_hits INTEGER,
    sacrifice_flies INTEGER,
    hit_by_pitch INTEGER,
    walks INTEGER,
    intentional_walks INTEGER,
    strikeouts INTEGER,
    stolen_bases INTEGER,
    caught_stealing INTEGER,
    grounded_into_double INTEGER,
    first_catcher_interference INTEGER,
    left_on_base INTEGER,
    pitchers_used INTEGER,
    individual_earned_runs INTEGER,
    team_earned_runs INTEGER,
    wild_pitches INTEGER,
    balks INTEGER,
    putouts INTEGER,
    assists INTEGER,
    errors INTEGER,
    passed_balls INTEGER,
    double_plays INTEGER,
    triple_plays INTEGER,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (league_id) REFERENCES league(league_id)
)
'''

c2 = '''
INSERT OR IGNORE INTO team_appearance
    SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays
    FROM game_log

UNION

    SELECT    
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    FROM game_log
'''

run_command(c1)
run_command(c2)

q = '''
    SELECT * FROM team_appearance
    WHERE game_id = "FW1187105040"
    ORDER By game_id, home
    '''

run_query(q)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,CL1,FW1187105040,0,None,0,000000000,30,4,1,0,0,0,0,0,0,1,None,6,1,None,-1,None,4,1,1,1,0,0,27,9,0,3,0,0
1,FW1,FW1187105040,1,None,2,010010000,31,4,1,0,0,2,0,0,0,1,None,0,0,None,-1,None,3,1,0,0,0,0,27,3,3,1,1,0


In [44]:
# create person appearance table

c = '''
    DROP TABLE IF EXISTS person_appearance'''
run_command(c)

c1 = '''
    CREATE TABLE IF NOT EXISTS person_appearance (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id,
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
    )'''

# umpire, managers, pitchers, rbi

c2 = '''
    INSERT INTO person_appearance (
        game_id,
        team_id,
        person_id,
        appearance_type_id
    )

        SELECT
            game_id,
            NULL,
            hp_umpire_id,
            "UHP"
        FROM game_log
        WHERE hp_umpire_id IS NOT NULL    

    UNION

        SELECT
            game_id,
            NULL,
            [1b_umpire_id],
            "U1B"
        FROM game_log
        WHERE "1b_umpire_id" IS NOT NULL

    UNION

        SELECT
            game_id,
            NULL,
            [2b_umpire_id],
            "U2B"
        FROM game_log
        WHERE [2b_umpire_id] IS NOT NULL

    UNION

        SELECT
            game_id,
            NULL,
            [3b_umpire_id],
            "U3B"
        FROM game_log
        WHERE [3b_umpire_id] IS NOT NULL

    UNION

        SELECT
            game_id,
            NULL,
            lf_umpire_id,
            "ULF"
        FROM game_log
        WHERE lf_umpire_id IS NOT NULL

    UNION

        SELECT
            game_id,
            NULL,
            rf_umpire_id,
            "URF"
        FROM game_log
        WHERE rf_umpire_id IS NOT NULL

    UNION

        SELECT
            game_id,
            v_name,
            v_manager_id,
            "MM"
        FROM game_log
        WHERE v_manager_id IS NOT NULL

    UNION

        SELECT
            game_id,
            h_name,
            h_manager_id,
            "MM"
        FROM game_log
        WHERE h_manager_id IS NOT NULL

    UNION

        SELECT
            game_id,
            CASE
                WHEN h_score > v_score THEN h_name
                ELSE v_name
                END,
            winning_pitcher_id,
            "AWP"
        FROM game_log
        WHERE winning_pitcher_id IS NOT NULL

    UNION

        SELECT
            game_id,
            CASE
                WHEN h_score < v_score THEN h_name
                ELSE v_name
                END,
            losing_pitcher_id,
            "ALP"
        FROM game_log
        WHERE losing_pitcher_id IS NOT NULL

    UNION

        SELECT
            game_id,
            CASE
                WHEN h_score > v_score THEN h_name
                ELSE v_name
                END,
            saving_pitcher_id,
            "ASP"
        FROM game_log
        WHERE saving_pitcher_id IS NOT NULL

    UNION

        SELECT
            game_id,
            CASE
                WHEN h_score > v_score THEN h_name
                ELSE v_name
                END,
            winning_rbi_batter_id,
            "AWB"
        FROM game_log
        WHERE winning_rbi_batter_id IS NOT NULL

    UNION

        SELECT
            game_id,
            v_name,
            v_starting_pitcher_id,
            "PSP"
        FROM game_log
        WHERE v_starting_pitcher_id IS NOT NULL

    UNION

        SELECT
            game_id,
            h_name,
            h_starting_pitcher_id,
            "PSP"
        FROM game_log
        WHERE h_starting_pitcher_id IS NOT NULL
    '''

run_command(c1)
run_command(c2)

# offense and defense
template = '''
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
'''

for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        # run commmand is a helper function which runs
        # a query against our database.
        run_command(template.format(**query_vars))

In [45]:
print(run_query("SELECT COUNT(DISTINCT game_id) as games_game FROM game"))
print(run_query("SELECT COUNT(DISTINCT game_id) as games_person_appearance FROM person_appearance"))

q = '''
SELECT
    pa.*,
    at.name,
    at.category
FROM person_appearance pa
INNER JOIN appearance_type at on at.appearance_type_id = pa.appearance_type_id
WHERE PA.game_id = (
                   SELECT max(game_id)
                    FROM person_appearance
                   )
ORDER BY team_id, appearance_type_id
'''

run_query(q)

   games_game
0      171907
   games_person_appearance
0                   171907


,appearance_id,person_id,team_id,game_id,appearance_type_id,name,category
0,1646114,steab101,None,WSU188409250,UHP,Home Plate,umpire
1,1646116,murnt101,BSU,WSU188409250,MM,Manager,manager
2,1646115,crane101,BSU,WSU188409250,PSP,Starting Pitcher,pitcher
3,1646118,scanm801,WSU,WSU188409250,MM,Manager,manager
4,1646117,dailh101,WSU,WSU188409250,PSP,Starting Pitcher,pitcher


In [46]:
# delete original datasets which are no longer needed

c1 = '''DROP TABLE IF EXISTS game_log'''
c2 = '''DROP TABLE IF EXISTS park_codes'''
c3 = '''DROP TABLE IF EXISTS team_codes'''
c4 = '''DROP TABLE IF EXISTS person_codes'''

run_command(c1)
run_command(c2)
run_command(c3)
run_command(c4)

show_tables()

,name,type
0,person,table
1,park,table
2,league,table
3,game,table
4,team,table
5,team_appearance,table
6,appearance_type,table
7,person_appearance,table
